In [1]:
from collections import defaultdict, Counter
import pandas as pd
from itertools import islice
from glob import glob
import os

# Parsing amino acid FASTA files

In [2]:
def parse_fasta_amino_acid(file_paths):
    """
    Parse multiple FASTA files to extract sequences along with their metadata.
    Virus type is extracted from the filename (e.g., 'mers-amino-acid.fasta' -> 'MERS').
    Returns a dictionary organized by virus type and protein type.
    """
    sequences = defaultdict(lambda: defaultdict(list))  # {virus_type: {protein_type: [sequences]}}
    
    for file_path in file_paths:
        virus_type = os.path.basename(file_path).split("-")[0].upper()
        
        with open(file_path, "r") as file:
            current_protein = None
            for line in file:
                line = line.strip()
                if line.startswith(">"):
                    # Parse protein name from header
                    header = line[1:]
                    parts = header.split("|")
                    protein_info = parts[1].split(" [")
                    current_protein = protein_info[0].strip()
                else:
                    # Append sequence fragment
                    sequences[virus_type][current_protein].append(line)
    return sequences

In [3]:
def compute_ngrams(sequence, n, is_amino_acid=True):
    """
    Compute all n-grams of length n from a given sequence,
    excluding n-grams with ambiguous characters.
    """
    if is_amino_acid:
        ambiguous_characters = {"B", "J", "O", "U", "X", "Z"}  # Ambiguous for amino acids
    else:
        ambiguous_characters = {"B", "D", "H", "K", "M", "R", "S", "V", "W", "Y", "N"}  # Ambiguous for nucleotides

    return [sequence[i:i+n] for i in range(len(sequence) - n + 1) 
            if not any(char in ambiguous_characters for char in sequence[i:i+n])]

In [10]:
def _norm(s: str) -> str:
    return str(s).strip().lower()

def generate_ngram_matrix_amino_acid_vertical(fasta_paths, n, mapping_csv):
    # 1) Load and clean CSV mapping
    mapping_df = pd.read_csv(mapping_csv, encoding="utf-8-sig").dropna(subset=["virus_type", "naziv", "dosadašnji nazivi"])
    mapping_df["virus_type_norm"] = mapping_df["virus_type"].map(_norm)
    mapping_df["naziv_norm"] = mapping_df["naziv"].map(_norm)

    # 2) Mapping by (virus, old_name) -> new_name (virus-specific!)
    mapping = {}
    for _, row in mapping_df.iterrows():
        virus = row["virus_type_norm"]
        new_name = row["naziv"].strip()  # keep formatted for display
        # split "dosadašnji nazivi" (old names)
        old_names = [x.strip() for x in str(row["dosadašnji nazivi"]).split(",") if x.strip()]
        # add all old names
        for old in old_names:
            mapping[(virus, old.lower())] = new_name
        # also add the "new" name as a possible old name (in case FASTA already uses it)
        mapping[(virus, row["naziv_norm"])] = new_name

    # 3) Parse FASTA (assumed to return dict: {virus: {protein: [seq1, seq2, ...]}})
    sequences = parse_fasta_amino_acid(fasta_paths)

    ngram_counts = []
    for virus_type, proteins in sequences.items():
        virus_norm = _norm(virus_type)
        for protein_type, seq_list in proteins.items():
            prot_norm = _norm(protein_type)

            # virus-specific mapping
            new_name = mapping.get((virus_norm, prot_norm))
            if not new_name:
                # no mapping for this protein in this virus -> skip
                continue

            combined_sequence = "".join(seq_list)
            ngrams = compute_ngrams(combined_sequence, n, is_amino_acid=True)
            for ng, count in Counter(ngrams).items():
                ngram_counts.append({
                    "ngram": ng,
                    "virus_type": virus_type.strip(),
                    "protein_type": new_name,  # mapped name from CSV
                    "count": count
                })

    if not ngram_counts:
        return pd.DataFrame(columns=["ngram"])

    df = pd.DataFrame(ngram_counts)

    # 4) Pivot: columns = (virus, new_name)
    ngram_matrix = df.pivot_table(
        index="ngram",
        columns=["virus_type", "protein_type"],
        values="count",
        aggfunc="sum",
        fill_value=0
    )

    # 5) (Optional but useful) – reindex columns to the exact expected ones from CSV
    #    This prevents "phantom" columns and ensures max number = defined in CSV
    allowed = (
        mapping_df
        .groupby("virus_type")["naziv"]
        .apply(lambda s: list(dict.fromkeys([x.strip() for x in s.tolist()])))
        .to_dict()
    )

    existing_viruses = [v for v in ngram_matrix.columns.get_level_values(0).unique() if v in allowed]
    desired_cols = []
    for v in existing_viruses:
        for pname in allowed[v]:
            desired_cols.append((v, pname))

    if desired_cols:
        ngram_matrix = ngram_matrix.reindex(columns=pd.MultiIndex.from_tuples(desired_cols, names=["virus_type", "protein_type"]), fill_value=0)

    # 6) Reset index so that 'ngram' becomes a column
    ngram_matrix = ngram_matrix.sort_index(axis=1)
    ngram_matrix.reset_index(inplace=True)

    return ngram_matrix

In [5]:
fasta_files = glob("data/*-amino-acid.fasta")
mapping_csv = "data/spisak_proteina.csv"  

ns = [3]
for n in ns:
    ngram_matrix = generate_ngram_matrix_amino_acid_vertical(fasta_files, n, mapping_csv)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_amino_acid_combined.csv", index=False)

ngram_matrix

virus_type   ngram                     EBOLA                                  \
protein_type       membrane glycoprotein - E nucleocapsid phosphoprotein - E   
0              AAA                        49                               9   
1              AAC                         0                               0   
2              AAD                         0                               0   
3              AAE                         0                              26   
4              AAF                         0                               0   
...            ...                       ...                             ...   
7498           YYS                         0                               0   
7499           YYT                         0                               0   
7500           YYV                         0                               0   
7501           YYW                         0                               0   
7502           YYY                         0                               5   

virus_type                                            \
protein_type polymerase - E surface glycoprotein - E   
0                        36                       11   
1                         0                        0   
2                       678                        9   
3                         4                      650   
4                       723                        9   
...                     ...                      ...   
7498                      8                      575   
7499                      0                        0   
7500                      0                        0   
7501                      0                        7   
7502                      0                        0   

virus_type                     MARBURG                                  \
protein_type membrane glycoprotein - R nucleocapsid phosphoprotein - R   
0                                    0                               0   
1                                    0                               0   
2                                    0                               0   
3                                    0                              89   
4                                    0                               0   
...                                ...                             ...   
7498                                 0                               0   
7499                                 0                               0   
7500                                 0                               0   
7501                                 0                               0   
7502                                 0                               0   

virus_type                                                MERS  ...  \
protein_type polymerase - R surface glycoprotein - R ORF1a - M  ...   
0                         0                        0      1762  ...   
1                         0                        0       580  ...   
2                        90                        0         3  ...   
3                         0                        0         6  ...   
4                        14                        0       521  ...   
...                     ...                      ...       ...  ...   
7498                      0                        0       591  ...   
7499                      0                        0         0  ...   
7500                      0                        0       586  ...   
7501                     90                        0         0  ...   
7502                      0                        0       582  ...   

virus_type                                                   \
protein_type envelope protein - M membrane glycoprotein - M   
0                               0                         0   
1                               0                         0   
2                               0                         0   
3                           

# Parsing nucleotide FASTA files


In [6]:
def parse_fasta_nucleotide(file_paths):
    """
    Parses multiple FASTA files and extracts virus types and sequences.
    Virus type is taken from the filename (e.g., 'mers-nucleotide.fasta' -> 'MERS').
    """
    virus_sequences = defaultdict(str)

    for file_path in file_paths:
        virus_type = os.path.basename(file_path).split("-")[0].upper()

        with open(file_path, 'r') as file:
            for line in file:
                if not line.startswith(">"):
                    virus_sequences[virus_type] += line.strip()

    return virus_sequences

In [7]:
def generate_ngram_matrix_nucleotide_vertical(fasta_paths, n):
    virus_sequences = parse_fasta_nucleotide(fasta_paths)
    ngram_counts = defaultdict(Counter)

    for virus_type, sequence in virus_sequences.items():
        ngrams = compute_ngrams(sequence, n, is_amino_acid=False)
        ngram_counts[virus_type].update(ngrams)

    all_ngrams = sorted(set(ngram for counter in ngram_counts.values() for ngram in counter))
    ngram_matrix = pd.DataFrame(index=all_ngrams, columns=sorted(ngram_counts.keys()), dtype=int).fillna(0)

    for virus_type, counter in ngram_counts.items():
        for ngram, count in counter.items():
            ngram_matrix.at[ngram, virus_type] = count

    ngram_matrix = ngram_matrix.astype(int)

    ngram_matrix.reset_index(inplace=True)
    ngram_matrix.rename(columns={"index": "ngram"}, inplace=True)

    return ngram_matrix

In [8]:
fasta_files = glob("data/*-nucleotide.fasta")
ns = [6, 7, 8] 
for n in ns:
    ngram_matrix = generate_ngram_matrix_nucleotide_vertical(fasta_files, n)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_nucleotide_combined.csv")
ngram_matrix

,ngram,EBOLA,MARBURG,MERS,SARS2
0,AAAAAAAA,30,0,3361,294
1,AAAAAAAC,1244,9,16,3
2,AAAAAAAG,4,0,975,6
3,AAAAAAAT,90,1,96,3
4,AAAAAACA,1560,129,2,0
...,...,...,...,...,...
63490,TTTTTTGT,357,33,13,107
63491,TTTTTTTA,30,0,21,5
63492,TTTTTTTC,278,0,3,1
63493,TTTTTTTG,25,5,9,111


In [9]:
fasta_files = glob("data/*-nucleotide.fasta")
ngram_matrix = generate_ngram_matrix_nucleotide_vertical(fasta_files, 9)
ngram_matrix.to_csv(f"csv_data/{9}gram_matrix_nucleotide_combined.csv")
ngram_matrix

,ngram,EBOLA,MARBURG,MERS,SARS2
0,AAAAAAAAA,15,0,2932,283
1,AAAAAAAAC,10,0,14,3
2,AAAAAAAAG,1,0,318,5
3,AAAAAAAAT,4,0,96,3
4,AAAAAAACA,605,0,2,0
...,...,...,...,...,...
195541,TTTTTTTGT,11,0,3,105
195542,TTTTTTTTA,1,0,0,2
195543,TTTTTTTTC,0,0,3,0
195544,TTTTTTTTG,1,0,4,111
